In [1]:
import numpy as np
eps = np.finfo(float).eps
from numpy import log2 as log
from pprint import pprint
import pandas as pd


In [24]:

dataset = {'A1':['young','old','old','old','old','adult','young','adult','old','young'],
       'A2':['married','married','married','single','married','single','single','married','single','married'],
       'A3':['small','small','big','big','big','small','small','small','small','big'],
       'y':['No','No','Yes','No','Yes','Yes','No','Yes','Yes','Yes']}


train = pd.DataFrame(dataset,columns=['A1','A2','A3','y'])
train.head()
train=pd.DataFrame({"Attrib1":["yes","no","no","yes","no","no","yes","no","no","no"],
                "Attrib2":["large","medium","small","medium","large","medium","large","small","medium","low"],
              "Attrib3":[125,100,70,120,95,60,220,85,75,90],
              "class":["no","no","no","no","yes","no","no","yes","no","yes"]},
              columns=["Attrib1","Attrib2","Attrib3","class"])
train

,Attrib1,Attrib2,Attrib3,class
0,yes,large,125,no
1,no,medium,100,no
2,no,small,70,no
3,yes,medium,120,no
4,no,large,95,yes
5,no,medium,60,no
6,yes,large,220,no
7,no,small,85,yes
8,no,medium,75,no
9,no,low,90,yes


In [25]:
class DecisionTree:
    
    def __init__(self,max_depth=20,depth=1):
        self.max_depth = max_depth
        self.depth = depth
        self.count=0
        
    
    def fit(self,df):
        # validate function pending 
        return self.BuildTree(df)
    
    
    
    def __find_entropy(self,df):
        # target 
        target = df.columns[-1] 
        #print(Class)

        entropy = 0
        # all varibles
        values = df[target].unique()
        #print(values)
        for value in values:
            percentage = df[target].value_counts()[value]/(len(df[target])+eps)
            #print(percentage)
            entropy += -percentage*np.log2(percentage+eps)
        return entropy
    
    def find_mean_value(self,variables):
        variables = np.sort(variables)
        return [variables[0]-5]+[(a + b) / 2 for a, b in zip(variables[0:], variables[1:])]+[variables[-1]+10]

    
    def find_entropy_col_continuos(self,df,col):
        # target value
        target = df.columns[-1]  
        #This gives all ouputs ('Yes' and 'No' )
        target_variables = df[target].unique()  
        #features value small ,medium ,large
        pre_variables = df[col].unique()
        variables =  self.find_mean_value(pre_variables)
        make_entropy_point =[]
        # make binary division 
        for variable in variables:
            #left tree #right tree index
            teamp_left,teamp_right = df[col]<=variable, df[col]>variable
            entropy2 =0
            for tree_side in (teamp_left,teamp_right):
                entropy =0
                total =np.sum(tree_side)
                for target_variable in target_variables:
                    #one class tupes # Yes class tuples or no class
                    teamp = df[tree_side][target] == target_variable
                    # number of elements in teamp 
                    #number of target_variable
                    ni=np.sum(teamp)
                    #print("ni {} class {}".format(ni,target_variable))
                    percentage = ni/(total+eps)
                    entropy += -percentage*log(percentage+eps)
                percentage2 = total/(len(df)+eps)
                entropy2 += percentage2*entropy
            make_entropy_point.append((entropy2))
        # return entropy and split point
        return np.min(make_entropy_point),variables[np.argmin(make_entropy_point)]


    def find_entropy_col_catogorical(self,df,col):
        # target value
        target = df.columns[-1]  
        #This gives all ouputs ('Yes' and 'No' )
        target_variables = df[target].unique()  
        #features value small ,medium ,large
        variables = df[col].unique()    
        entropy2 = 0
        for variable in variables:
            entropy = 0
            for target_variable in target_variables:
                col_f = df[col]==variable
                ni = len(df[col][col_f][df[target] ==target_variable])
                total = len(df[col][col_f])
                percentage = ni/(total+eps)
                entropy += -percentage*log(percentage+eps)
            percentage2 = total/(len(df)+eps)
            entropy2 += percentage2*entropy
        return entropy2


    def best_node(self,df):
        information_gain = []
        # for continuos  split  used
        split=None
        inf_g=0
        for col in df.columns[:-1]:
            if len(df[col].unique())>4 and isinstance(df[col].unique()[0],(np.int64,int,float)): 
                inf,split=self.find_entropy_col_continuos(df,col)
                information_gain.append((inf_g,split))
            else:
                inf_g=self.find_entropy_col_catogorical(df,col)
                information_gain.append((inf_g,None))

            #print(key)
            # choice col with max infomation gain
        inf_g_x ,split_x =zip(*information_gain)
        
        max_inf_gain_col=np.argmin(inf_g_x)

        return df.columns[:-1][max_inf_gain_col],split_x[max_inf_gain_col]
    
    # for continuous value 
    '''    
    def best_split(s1,s2):
        
        return DecisionTree.__find_entropy(s1)'''
    def BuildTree(self,df,tree=None): 
        #Here we build our decision tree
        #print(self.depth)
        #print(self.max_depth)
        if self.depth >= self.max_depth:
            clValue,counts = np.unique(df[df.columns[-1]],return_counts=True)
            return clValue[np.argmax(counts)]


        #Get attribute with maximum information gain
        node,split= self.best_node(df)
        #print(node)

        #Get distinct value of that attribute
        #['large' 'low' 'medium' 'large'] example
        col_f = np.unique(df[node])
        #print(attValue)

        #dictionary for store tree   
        if tree is None:                    
            tree={}
            # first node  
            tree[node] = {}
   
       # recursively call to construct the tree  
       # if int split by best mid point   # pending work   
        #height of tree increase by 1    
        self.depth=self.depth+1
        #print(split)
        
        # for continuous value
        if split is not None:
            left_split = df[df[node] <= split].reset_index(drop=True)
            right_split = df[df[node] > split].reset_index(drop=True)
            #print(right_split)
            for subtable in (left_split,right_split):
                if len(subtable)==0:
                    continue
                
                clValue,counts = np.unique(subtable[ df.columns[-1]],return_counts=True)
                # check subset is pure or not 
                #print(clValue)

                value=split
                if len(counts)==1:
                    # leaf node
                    tree[node][value] = clValue[0]
                else:
                    tree[node][value]= self.BuildTree(subtable) 
                # (less then or equal 97)  or more then 97 
                value=value+1
                
            
        else:# for catogorical value
            for value in col_f:
                subtable = df[df[node] == value].reset_index(drop=True)
                clValue,counts = np.unique(subtable[ df.columns[-1]],return_counts=True)
                # check subset is pure or not 
                
                if len(counts)==1:
                    # leaf node
                    tree[node][value] = clValue[0]                                                    
                else:
                    # recursion 
                    tree[node][value] = self.BuildTree(subtable) 

        return tree
    
    def predict(self,inst,tree):
        #This function is used to predict for any input variable 
        # take node of tree 
        
        for nodes in tree.keys():        
            # values of node  'large' 'low' 'medium' 'small' 
            value = inst[nodes]
            # for contineous value 
            # value if integer
            
            branches=list(tree[nodes].keys())
            
            if  isinstance(branches[0], (np.int64,int,np.float64)):
                if len(branches)==1 and isinstance(tree[nodes][branches[0]],(np.int64,int,np.float64)):
                                                               return tree[nodes][branches[0]]
                
                if branches[0]<=inst[nodes]:
                    tree=tree[nodes][branches[0]]
                else:
                    try:
                        tree=tree[nodes][branches[1]]
                    except:
                        tree=tree[nodes][branches[0]]
            else:
                try:
                    value = inst[nodes]
                    tree = tree[nodes][value]
                    # handling mising value  example small attribute does not exit in test dataset 
                except:
                    # find all keys
                    All_braches =tree[nodes].keys()
                    
                    results=[]
                    tree = tree[nodes]
                    
                    for brach_i in All_braches:
                        #pure node
                        sub_tree=tree[brach_i]
                        if type(sub_tree) is dict:
                            results.append(self.predict(inst,sub_tree))
                        else:
                            results.append(sub_tree)
                        
                    # most frequent string elemnt in list 
                    unique,pos = np.unique(results,return_inverse=True) #Finds all unique elements and their positions
                    counts = np.bincount(pos)                     #Count the number of each unique element
                    maxpos = counts.argmax()                      #Finds the positions of the maximum count
                    prediction = unique[maxpos]
                    results =[]
                    return unique[maxpos]
            prediction = 0

            if type(tree) is dict:
                prediction = self.predict(inst, tree)
            else:
                prediction = tree
                break;                            

        return prediction
    
    def prediction(self,inst,tree):
        prediction =[]
        for i in range(len(inst)):
            prediction.append(self.predict(inst.iloc[i],tree))
        return prediction
    
    def evaluate(self,X,y):
        a=sum(X==y)
        print('accuracy = {} %'.format(float(a/(len(X)+eps)*100)))
        #return float(a/(len(x)+eps)*100)
        

* input dataframe with lables

In [26]:
train = pd.DataFrame({"name":["porcupine","cat","bat","Whale","Salamander","Komodo dragon","Python","salmon","Eagle","Guppy"],
                   "Body_teparature":["Warm-blooded","Warm-blooded","Warm-blooded","Warm-blooded","cold-blooded","cold-blooded","cold-blooded","cold-blooded","Warm-blooded","cold-blooded"],
                    "gives_Birth":["Yes","Yes","Yes","Yes","no","no","no","no","no","yes"],
                    "Four-legged":["Yes","Yes","no","no","Yes","Yes","no","no","no","no"],
                    "Hibernates":["Yes","no","Yes","no","Yes","no","Yes","no","no","no"],
                    "y":["Yes","Yes","no","no","no","no","no","no","no","no"]})

test = pd.DataFrame({"name":["Human","Pigeon","Elepheant","Leopard shark","Turtle","penguin","Eel","Doplpin","spiny anteater","Gilla monster"],
                   "Body_teparature":["Wa rm-blooded","Warm-blooded","Warm-blooded","cold-blooded","cold-blooded","cold-blooded","cold-blooded","Warm-blooded","Warm-blooded","cold-blooded"],
                    "gives_Birth":["Yes","no","Yes","Yes","no","no","no","Yes","no","no"],
                    "Four-legged":["no","no","Yes","no","Yes","no","no","no","Yes","Yes"],
                    "Hibernates":["no","no","no","no","no","no","no","no","Yes","Yes"],
                    "y":["Yes","no","Yes","no","no","no","no","Yes","Yes","no"]})

In [27]:
model =DecisionTree(depth=1,max_depth=10)
tree = model.fit(train)
label=train[train.columns[-1]]
predict=model.prediction(train[train.columns[:-1]],tree)
model.evaluate(label,predict)

accuracy = 100.0 %


In [28]:
tree

{'name': {'Eagle': 'no',
  'Guppy': 'no',
  'Komodo dragon': 'no',
  'Python': 'no',
  'Salamander': 'no',
  'Whale': 'no',
  'bat': 'no',
  'cat': 'Yes',
  'porcupine': 'Yes',
  'salmon': 'no'}}

## Randome forest tree

* input dataframe with lables

**Bootstrapped dataset**


In [29]:
class RandomForest:
    
    def __init__(self,ensemble=3):
        self.ensemble=ensemble
        self.store_all_tree =[]
        self.features_name= None
        self.low =0
        self.high=0
        
    def fit(self,train):
        self.store_all_tree =[]
        self.features_name= train.columns[:-1]
        # number of tuples 
        self.low = train.first_valid_index() # min
        self.high = train.last_valid_index() #max
        for i in range(self.ensemble):
            Decision_tree = DecisionTree()
            # create tree with for diffrent datasets 
            tree =Decision_tree.fit(self.Bootstrapped(train))
            # store tree
            self.store_all_tree.append(tree)
        return self.store_all_tree
        
    def Bootstrapped(self,train):
        
        #Return random integers from `low` (inclusive) to `high` (exclusive).
        index =np.random.randint(low=self.low,high=self.high+1,size=10)
        # new traing data only row with replacement all columns
        new_train = train.iloc[index].reset_index(drop=True)
        # number of features choice randomly m<<M(total size of features)

        #name and size of features
        size_of_col=len(self.features_name) 
        
        # number of cloumns pick min 1 col max size_of_col(all)
        # min 2 col
        number_of_col =  np.random.randint(low=2,high=size_of_col+1,size=1)
        # if number_of_col 
        # slectect number_of_col number of  columns pick randomly 
        col_val =np.random.choice(range(size_of_col), number_of_col, replace=False)

        col_name =self.features_name[col_val] 


        # new dataset with m columns
        new_train = new_train[col_name]
        # insert label columns
        
        new_train[train.columns[-1]]=train[train.columns[-1]]

        return new_train
    
    def vote(self,predict):
        predict =np.array(predict).T
        value=[]
        for row in range(predict.shape[0]):
            unique,pos = np.unique(predict[row],return_inverse=True) #Finds all unique elements and their positions
            counts = np.bincount(pos)                     #Count the number of each unique element
            maxpos = counts.argmax()                      #Finds the positions of the maximum count
            value.append(unique[maxpos])

        return value
        
    def prediction(self,x_test):
        predict =[]
        for i in range(self.ensemble):
            Decision_tree=DecisionTree()
            predict.append(Decision_tree.prediction(x_test,self.store_all_tree[i]))
        return self.vote(predict)
  
    def evaluate(self,X,y):
        a=sum(X==y)
        print('accuracy = {} %'.format(float(a/(len(X)+eps)*100)))
        #return float(a/(len(x)+eps)*100)
        
        

In [30]:
model = RandomForest(ensemble=6)
store_all_tree=model.fit(train)
predict=model.prediction(train[train.columns[:-1]])
model.evaluate(predict,train[train.columns[-1]])

accuracy = 70.0 %


In [31]:
# accuracy of randome forest is less 
# maybe due to small dataset
# 


In [32]:
pprint(store_all_tree)

[{'name': {'Eagle': 'no',
           'Whale': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': {'Four-legged': {'no': 'no'}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}},
           'cat': 'no',
           'porcupine': 'Yes'}},
 {'Hibernates': {'Yes': 'no',
                 'no': {'Body_teparature': {'Warm-blooded': {'Hibernates': {'no': {'Hibernates': {'no': {'Hibernates': {'no': {'Hibernates': {'no': {'Hibernates': {'no': {'Hibernates': {'no': {'Hibernates': {'no': {'Hibernates': {'no': {'Hibernates': {'no': {'Hibernates': {'no': {'Hibernates': {'no': {'Hibernates': {'no': {'Hibernates': {'no': {'Hibernates': {'no': {'Hibernates': {'no': {'Hibernates': {'no':